In [ ]:
"R.01/97.956 Data: 15/maio/1989 Pelo formal de partilha passado em 09 de fevereiro de 1982, pelo Cartório do 6.º Ofício ou 2. de Direito da 8.a Vara, ambos da Família e Sucessões desta Capital, o Espólio de SIMON KUCZYNSKI -(falecido em 20 de abril de 1973, no estado civil de casado pelo regime da comunhão de bens com Elza Nazareth Kuczynski) transmitiu por partilha a 1) ELZA NAZARETH KUCZYNSKI, brasileira, viúva, do lar, RG. 584.724, CIC. n.º 000.814.868-63, domiciliada nesta Capital, à Rua Dr. Numa Pereira do Vale, 272; 2) RUTH NAZARETH KUCZYNSKI, brasileira, solteira, maior, estudante, RG. n.º 5.333.362, CIC. sob o n.º 033.850.628-45, domiciliada nesta Capital, à Rua Numa Pereira do Vale, 272; 3) PAULO NAZARETH KUCZYNSKI, brasileiro, solteiro, maior, proprietário, RG. 3.951.101, C.I.C.M. F. sob n.º 531.813.108-59, domiciliado nesta Capital, à Rua Numa Pereira do Vale, 272; e, 4) SERGIO NAZARETH KUCZYNSKI, brasileiro, solteiro, maior, estudante, CIC. 034.354.838-06, domiciliado nesta Capital, à Rua Numa Pereira do Vale, 272, o imóvel, pelo valor de Cr$51.656,00, na proporção de 1/2 à viúva; e, 1/6 a cada um dos demais herdeiros.-"

In [ ]:
#SIMON KUCZYNSKI
{"LedgerEntryID": 1, "DeedID": "DEED001", "OwnerName": "SIMON KUCZYNSKI", "ChangeInSharePercentage": -1.00, "EffectiveDate": "1989-05-15", "DocRef": "R.1"},

#ELZA NAZARETH KUCZYNSKI
{"LedgerEntryID": 2, "DeedID": "DEED001", "OwnerName": "ELZA NAZARETH KUCZYNSKI", "ChangeInSharePercentage": 0.5, "EffectiveDate": "1989-05-15", "DocRef": "R.1"},
#RUTH NAZARETH KUCZYNSKI
{"LedgerEntryID": 3, "DeedID": "DEED001", "OwnerName": "RUTH NAZARETH KUCZYNSKI", "ChangeInSharePercentage": 0.1666, "EffectiveDate": "1989-05-15", "DocRef": "R.1"},
#PAULO NAZARETH KUCZYNSKI
{"LedgerEntryID": 4, "DeedID": "DEED001", "OwnerName": "PAULO NAZARETH KUCZYNSKI", "ChangeInSharePercentage": 0.1666, "EffectiveDate": "1989-05-15", "DocRef": "R.1"},
#SERGIO NAZARETH KUCZYNSKI
{"LedgerEntryID": 5, "DeedID": "DEED001", "OwnerName": "SERGIO NAZARETH KUCZYNSKI", "ChangeInSharePercentage": 0.1666, "EffectiveDate": "1989-05-15", "DocRef": "R.1"},


In [19]:
import re
import json

def extract_transfers(text, starting_id=1, deed_id="DEED001", doc_ref="R.1"):
    # Extract effective date from the text using a regex pattern.
    date_match = re.search(r'Data:\s*(\d{1,2})/(\w+)/(\d{4})', text, re.IGNORECASE)
    if date_match:
        day, month_pt, year = date_match.groups()
        month_map = {
            'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04',
            'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08',
            'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
        }
        month = month_map.get(month_pt.lower(), '01')
        effective_date = f"{year}-{month}-{int(day):02d}"
    else:
        effective_date = None

    # Use regex to extract owner names.
    # This pattern assumes names are in uppercase and preceded by numbers like "1)" or "2)".
    owner_pattern = r'\d+\)\s*([A-Z\s]+?)(?=,|;|$)'
    owners = re.findall(owner_pattern, text)

    transfers = []
    ledger_entry_id = starting_id

    if owners:
        # First entry: the transferor (e.g., Simon) losing 100% of his share.
        transfers.append({
            "LedgerEntryID": ledger_entry_id,
            "DeedID": deed_id,
            "OwnerName": "SIMON KUCZYNSKI",  # This would be extracted with a separate pattern in a real case.
            "ChangeInSharePercentage": -1.00,
            "EffectiveDate": effective_date,
            "DocRef": doc_ref
        })
        ledger_entry_id += 1
        
        # For a 1-to-many transfer, distribute the remaining share equally.
        if len(owners) > 0:
            share_per_recipient = round(1.00 / len(owners), 4)
            for owner in owners:
                transfers.append({
                    "LedgerEntryID": ledger_entry_id,
                    "DeedID": deed_id,
                    "OwnerName": owner.strip(),
                    "ChangeInSharePercentage": share_per_recipient,
                    "EffectiveDate": effective_date,
                    "DocRef": doc_ref
                })
                ledger_entry_id += 1

    return transfers

# Example usage:
text = (
    "R.01/97.956 Data: 15/maio/1989 Pelo formal de partilha passado em 09 de fevereiro de 1982, "
    "pelo Cartório do 6.º Ofício ou 2. de Direito da 8.a Vara, ambos da Família e Sucessões desta Capital, "
    "o Espólio de SIMON KUCZYNSKI -(falecido em 20 de abril de 1973, no estado civil de casado pelo regime da comunhão de bens com Elza Nazareth Kuczynski) "
    "transmitiu por partilha a 1) ELZA NAZARETH KUCZYNSKI, brasileira, viúva, do lar, RG. 584.724, CIC. n.º 000.814.868-63, domiciliada nesta Capital, à Rua Dr. Numa Pereira do Vale, 272; "
    "2) RUTH NAZARETH KUCZYNSKI, brasileira, solteira, maior, estudante, RG. n.º 5.333.362, CIC. sob o n.º 033.850.628-45, domiciliada nesta Capital, à Rua Numa Pereira do Vale, 272; "
    "3) PAULO NAZARETH KUCZYNSKI, brasileiro, solteiro, maior, proprietário, RG. 3.951.101, C.I.C.M. F. sob n.º 531.813.108-59, domiciliado nesta Capital, à Rua Numa Pereira do Vale, 272; "
    "e, 4) SERGIO NAZARETH KUCZYNSKI, brasileiro, solteiro, maior, estudante, CIC. 034.354.838-06, domiciliado nesta Capital, à Rua Numa Pereira do Vale, 272, "
    "o imóvel, pelo valor de Cr$51.656,00, na proporção de 1/2 à viúva; e, 1/6 a cada um dos demais herdeiros.-"
)

result = extract_transfers(text)
print(json.dumps(result, indent=2, ensure_ascii=False))


[
  {
    "LedgerEntryID": 1,
    "DeedID": "DEED001",
    "OwnerName": "SIMON KUCZYNSKI",
    "ChangeInSharePercentage": -1.0,
    "EffectiveDate": "1989-05-15",
    "DocRef": "R.1"
  },
  {
    "LedgerEntryID": 2,
    "DeedID": "DEED001",
    "OwnerName": "ELZA NAZARETH KUCZYNSKI",
    "ChangeInSharePercentage": 0.25,
    "EffectiveDate": "1989-05-15",
    "DocRef": "R.1"
  },
  {
    "LedgerEntryID": 3,
    "DeedID": "DEED001",
    "OwnerName": "RUTH NAZARETH KUCZYNSKI",
    "ChangeInSharePercentage": 0.25,
    "EffectiveDate": "1989-05-15",
    "DocRef": "R.1"
  },
  {
    "LedgerEntryID": 4,
    "DeedID": "DEED001",
    "OwnerName": "PAULO NAZARETH KUCZYNSKI",
    "ChangeInSharePercentage": 0.25,
    "EffectiveDate": "1989-05-15",
    "DocRef": "R.1"
  },
  {
    "LedgerEntryID": 5,
    "DeedID": "DEED001",
    "OwnerName": "SERGIO NAZARETH KUCZYNSKI",
    "ChangeInSharePercentage": 0.25,
    "EffectiveDate": "1989-05-15",
    "DocRef": "R.1"
  }
]


In [12]:
import json
from collections import defaultdict
from datetime import datetime
import graphviz # pip install graphviz

# --- 1. Load Data ---
# Assume the ledger data for the single property is loaded:
# with open('prop123_ledger.json', 'r') as f:
#     ledger_data = json.load(f)
# Example data:
ledger_data = [
    {"LedgerEntryID": 1, "DeedID": "DEED001", "OwnerName": "Origin", "ChangeInSharePercentage": -1.00, "EffectiveDate": "2010-01-15", "DocRef": "Matricula 999 / R.1"},
    {"LedgerEntryID": 2, "DeedID": "DEED001", "OwnerName": "Marco", "ChangeInSharePercentage": 1.00, "EffectiveDate": "2010-01-15", "DocRef": "Matricula 999 / R.1"},

    {"LedgerEntryID": 3, "DeedID": "DEED002", "OwnerName": "Marco", "ChangeInSharePercentage": -0.60, "EffectiveDate": "2015-05-20", "DocRef": "Matricula 999 / R.2"},
    {"LedgerEntryID": 4, "DeedID": "DEED002", "OwnerName": "Carlos", "ChangeInSharePercentage": 0.60, "EffectiveDate": "2015-05-20", "DocRef": "Matricula 999 / R.2"},

    {"LedgerEntryID": 5, "DeedID": "DEED003", "OwnerName": "Carlos", "ChangeInSharePercentage": -0.40, "EffectiveDate": "2017-08-10", "DocRef": "Matricula 999 / R.3"},
    {"LedgerEntryID": 6, "DeedID": "DEED003", "OwnerName": "Laura", "ChangeInSharePercentage": 0.20, "EffectiveDate": "2017-08-10", "DocRef": "Matricula 999 / R.3"},
    {"LedgerEntryID": 7, "DeedID": "DEED003", "OwnerName": "Maria", "ChangeInSharePercentage": 0.20, "EffectiveDate": "2017-08-10", "DocRef": "Matricula 999 / R.3"},

    {"LedgerEntryID": 8, "DeedID": "DEED004", "OwnerName": "Maria", "ChangeInSharePercentage": -0.10, "EffectiveDate": "2018-08-10", "DocRef": "Matricula 999 / R.4"},
    {"LedgerEntryID": 9, "DeedID": "DEED004", "OwnerName": "Marco", "ChangeInSharePercentage": 0.10, "EffectiveDate": "2018-08-10", "DocRef": "Matricula 999 / R.4"},
]


In [13]:
# Previous imports assumed (json, defaultdict, datetime, graphviz)
# Assume ledger_data is loaded as before

# Function to calculate final status (needed for styling nodes)
def get_final_ownership_status(ledger):
    """Calculates final ownership percentages by summing all ledger entries."""
    ownership = defaultdict(float)
    for entry in ledger:
        ownership[entry['OwnerName']] += entry['ChangeInSharePercentage']
    return {name: round(share * 100, 2) for name, share in ownership.items() if abs(share) > 1e-9}

def visualize_full_flow_vertical(ledger, current_owners, filename="full_ownership_flow_vertical"):
    """Generates a VERTICAL Graphviz diagram showing all owners and transfers."""
    dot = graphviz.Digraph(comment='Full Ownership Flow - Vertical')
    dot.attr('node', shape='box', style='rounded')
    # --- Changed rankdir here ---
    dot.attr('graph', rankdir='TB', splines='true', overlap='false') # Top-to-Bottom layout

    owner_nodes = set()
    deed_nodes = set()

    # Sort ledger by date for somewhat chronological layout processing
    ledger.sort(key=lambda x: datetime.strptime(x['EffectiveDate'], '%Y-%m-%d').date())

    # Define nodes for owners and deeds
    for entry in ledger:
        owner_name = entry['OwnerName']
        if owner_name not in owner_nodes:
             fillcolor = 'lightblue' if owner_name in current_owners else 'lightgrey'
             dot.node(owner_name, owner_name, style='filled', fillcolor=fillcolor)
             owner_nodes.add(owner_name)

        deed_id = entry['DeedID']
        if deed_id not in deed_nodes:
            deed_label = f"{entry.get('DocRef', deed_id)}\n({entry['EffectiveDate']})"
            dot.node(deed_id, deed_label, shape='diamond', style='filled', fillcolor='beige')
            deed_nodes.add(deed_id)

    # Create edges: Owner -> Deed (giving), Deed -> Owner (receiving)
    for entry in ledger:
        owner_name = entry['OwnerName']
        deed_id = entry['DeedID']
        share_change_pct = round(entry['ChangeInSharePercentage'] * 100, 2)

        if share_change_pct < 0:
            dot.edge(owner_name, deed_id, label=f"{share_change_pct}%")
        elif share_change_pct > 0:
            dot.edge(deed_id, owner_name, label=f"+{share_change_pct}%")

    try:
        dot.render(filename, view=True, format='png', cleanup=True)
        print(f"Vertical flow graph saved as {filename}.png and opened.")
    except Exception as e:
        print(f"Could not render graph. Is Graphviz installed and in PATH? Error: {e}")
        # print("Graphviz DOT source:\n", dot.source) # Uncomment to debug DOT source

In [10]:

# --- Example Usage ---
# 1. Calculate final status
final_ownership = get_final_ownership_status(ledger_data)
print(f"\nFinal Ownership Status (as of {datetime.now().date()}):")
for owner, share in final_ownership.items():
    print(f"- {owner}: {share}%")
print("-" * 20)

# 2. Visualize the full flow
visualize_full_flow_vertical(ledger_data, final_ownership.keys(), filename="property_flow_diagram")


Final Ownership Status (as of 2025-04-02):
- Original Seller Corp.: -100.0%
- Marco: 40.0%
- Carlos: 20.0%
- Laura: 20.0%
- Maria: 20.0%
--------------------
Vertical flow graph saved as property_flow_diagram.png and opened.


In [18]:
# Imports assumed: json, defaultdict, datetime, timedelta, go, pd
import plotly.graph_objects as go
import pandas as pd
from collections import defaultdict
from datetime import datetime, timedelta

# Assume ledger_data is loaded

# Helper function (same as before)
def get_ownership_at_date(ledger, target_date):
    """Calculates ownership state just AFTER a given date."""
    ownership = defaultdict(float)
    relevant_entries = [
        entry for entry in ledger
        if datetime.strptime(entry['EffectiveDate'], '%Y-%m-%d').date() <= target_date
    ]
    for entry in relevant_entries:
        ownership[entry['OwnerName']] += entry['ChangeInSharePercentage']
    # Return shares as fractions (0.0 to 1.0)
    return {name: round(share, 6) for name, share in ownership.items() if abs(share) > 1e-9}

def visualize_sankey_flow(ledger, filename="sankey_flow.html"):
    """Generates an interactive Plotly Sankey diagram showing ownership flow."""

    if not ledger:
        print("Ledger data is empty.")
        return

    # --- 1. Prepare Data ---
    # Get unique sorted dates from actual transactions
    dates = sorted(list(set(datetime.strptime(e['EffectiveDate'], '%Y-%m-%d').date() for e in ledger)))

    if not dates:
        print("Ledger contains no valid dates.")
        return

    # Define the time points for our states
    start_date = dates[0] - timedelta(days=1) # Pseudo-date before first transaction
    all_dates = [start_date] + dates

    # Calculate initial state explicitly based on first deed's grantors
    initial_state = defaultdict(float)
    first_date = dates[0]
    first_deed_entries = [e for e in ledger if datetime.strptime(e['EffectiveDate'], '%Y-%m-%d').date() == first_date]
    initial_total_share = 0.0
    processed_initial_grantors = set()
    for entry in first_deed_entries:
         # Ensure we only count each grantor's initial contribution once if multiple entries exist
         if entry['ChangeInSharePercentage'] < -1e-9 and entry['OwnerName'] not in processed_initial_grantors:
              share = abs(entry['ChangeInSharePercentage'])
              initial_state[entry['OwnerName']] = share
              initial_total_share += share
              processed_initial_grantors.add(entry['OwnerName'])

    # If initial grantors don't sum to 100%, add a placeholder "SOURCE" if needed
    if abs(initial_total_share - 1.0) > 1e-9:
         print(f"Warning: Initial grantors on {first_date} account for {initial_total_share*100:.2f}%. Assuming remainder came from 'SOURCE'.")
         # To make it sum to 1, we could add a SOURCE node, but let's proceed assuming the ledger is complete/correct for the first transaction.
         # If initial_state is empty (e.g., first entry is just positive), need a source.
         if not initial_state:
             initial_state["SOURCE"] = 1.0 # Assume property appeared from nowhere

    # Calculate ownership state after each relevant date
    states = {start_date: dict(initial_state)} # Start with the calculated initial state
    for d in dates:
         states[d] = get_ownership_at_date(ledger, d)

    # --- Create Nodes ---
    internal_node_ids = [] # Stores unique Owner@Date identifiers
    node_map_to_index = {} # Maps Owner@Date to integer index
    node_display_labels = [] # Stores Owner (Share%) labels for display
    node_colors = [] # Stores colors based on owner

    # Get all unique owner names across all states for consistent coloring
    all_owners_ever = set(owner for state in states.values() for owner in state)
    # Assign a color to each unique owner
    owner_color_map = {name: f'hsl({i * 360 / len(all_owners_ever)}, 70%, 70%)' for i, name in enumerate(sorted(list(all_owners_ever)))}
    # Add color for the SOURCE node if it exists
    if "SOURCE" in initial_state:
        owner_color_map["SOURCE"] = 'lightgrey'

    # Iterate through dates and states to define nodes
    for date in all_dates:
        date_str = date.strftime('%Y-%m-%d')
        state = states[date]
        for owner, share in state.items():
            if share > 1e-9: # Only create nodes for owners holding shares
                internal_id = f"{owner}@{date_str}"
                if internal_id not in node_map_to_index:
                    # Map internal ID to index
                    node_map_to_index[internal_id] = len(internal_node_ids)
                    internal_node_ids.append(internal_id)
                    # Create display label with percentage at this point
                    display_label = f"{owner} ({share*100:.1f}%)"
                    node_display_labels.append(display_label)
                    # Assign color
                    node_colors.append(owner_color_map.get(owner, 'grey'))

    # --- Create Links ---
    link_source = []
    link_target = []
    link_value = []
    link_label = [] # Hover text

    # Iterate through time periods (date_prev -> date_curr)
    for i in range(len(all_dates) - 1):
        date_prev = all_dates[i]
        date_curr = all_dates[i+1]
        date_prev_str = date_prev.strftime('%Y-%m-%d')
        date_curr_str = date_curr.strftime('%Y-%m-%d')

        state_prev = states[date_prev]
        state_curr = states[date_curr]

        # Deeds effective on date_curr
        deeds_at_curr = [e for e in ledger if datetime.strptime(e['EffectiveDate'], '%Y-%m-%d').date() == date_curr]

        # Process flow for each owner holding shares previously
        for owner, share_prev in state_prev.items():
            if share_prev < 1e-9: continue

            internal_id_prev = f"{owner}@{date_prev_str}"
            # Ensure the source node exists in our map (it should if share_prev > 0)
            if internal_id_prev not in node_map_to_index: continue
            source_idx = node_map_to_index[internal_id_prev]

            share_curr = state_curr.get(owner, 0.0) # Owner's share *after* date_curr transactions

            # A: Persistence Flow (Share kept by the owner)
            persisting_share = min(share_prev, share_curr)
            if persisting_share > 1e-9:
                 internal_id_curr = f"{owner}@{date_curr_str}"
                 # Ensure target node exists (it should if share_curr > 0)
                 if internal_id_curr in node_map_to_index:
                    target_idx = node_map_to_index[internal_id_curr]
                    link_source.append(source_idx)
                    link_target.append(target_idx)
                    link_value.append(persisting_share * 100)
                    link_label.append(f"{owner} keeps {persisting_share*100:.1f}%")

            # B: Outgoing Transfer Flow (Share transferred *out* by this owner)
            # Calculate based on difference OR by finding grantor entries
            transferred_share_calculated = share_prev - persisting_share
            if transferred_share_calculated > 1e-9:
                # Find deeds where this owner was a grantor on date_curr
                grantor_deeds_entries = [d for d in deeds_at_curr if d['OwnerName'] == owner and d['ChangeInSharePercentage'] < -1e-9]
                processed_grantor_share = 0.0 # Track share processed from this grantor node

                for deed_entry in grantor_deeds_entries:
                    deed_id = deed_entry['DeedID']
                    share_given_by_owner_in_deed = abs(deed_entry['ChangeInSharePercentage'])

                    # Find grantees in the SAME deed
                    grantees_in_deed = [g for g in deeds_at_curr if g['DeedID'] == deed_id and g['ChangeInSharePercentage'] > 1e-9]

                    # Distribute the share_given_by_owner proportionally or directly?
                    # Let's link based on grantee's gain in that specific deed
                    for grantee_entry in grantees_in_deed:
                        grantee_name = grantee_entry['OwnerName']
                        share_gained_by_grantee = grantee_entry['ChangeInSharePercentage']
                        internal_id_grantee_curr = f"{grantee_name}@{date_curr_str}"

                        # Ensure grantee target node exists
                        if internal_id_grantee_curr in node_map_to_index:
                            target_idx = node_map_to_index[internal_id_grantee_curr]
                            # Value is the specific share gained by the grantee in this deed
                            # This assumes the ledger correctly pairs losses/gains per deed
                            link_value_transfer = share_gained_by_grantee * 100
                            # Ensure we don't over-allocate from the source node if multiple grantors exist?
                            # For now, link Grantor -> Grantee with the value Grantee gained.
                            link_source.append(source_idx)
                            link_target.append(target_idx)
                            link_value.append(link_value_transfer)
                            link_label.append(f"{owner} -> {grantee_name} ({deed_entry.get('DocRef', deed_id)}): +{link_value_transfer:.1f}%")
                            processed_grantor_share += share_gained_by_grantee # Track how much flow we created

                # Note: If multiple grantors contributed to a single grantee gain, this logic
                # might create multiple links pointing to the grantee, which is correct for Sankey.
                # A potential check: ensure sum(link_value for links originating from source_idx)
                # approximately equals share_prev * 100.


    ## --- 2. Create Plotly Figure ---
    #if not node_labels:
    #    print("No nodes generated for Sankey diagram.")
    #    return
        
    fig = go.Figure(data=[go.Sankey(
        arrangement = "snap", # Align nodes vertically
        node = dict(
          pad = 25, # Increase padding
          thickness = 15,
          line = dict(color = "black", width = 0.5),
          label = node_display_labels, # Use the formatted labels
          color = node_colors, # Use assigned colors
          # x = node_x, # Optional manual positioning
          # y = node_y,
        ),
        link = dict(
          source = link_source,
          target = link_target,
          value = link_value,
          label = link_label, # Add hover labels
          color = 'rgba(200,200,200,0.5)' # Light grey links
      ))])

    fig.update_layout(
        title_text="Property Ownership Flow (Sankey)",
        font=dict(size=10, color='black'),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )


    try:
        fig.write_html(filename)
        print(f"Sankey diagram saved as {filename}")
        # fig.show() # Uncomment to automatically open in browser
    except Exception as e:
        print(f"Could not save Sankey diagram. Error: {e}")

# --- Example Usage ---
# Assume ledger_data is loaded

# Generate Sankey Diagram
visualize_sankey_flow(ledger_data, filename="ownership_sankey_final.html")

Sankey diagram saved as ownership_sankey_final.html


In [16]:
# Additional imports needed: pip install plotly pandas
import plotly.graph_objects as go
import pandas as pd

# Assume ledger_data is loaded as before

def get_ownership_at_date(ledger, target_date):
    """Calculates ownership state just AFTER a given date."""
    ownership = defaultdict(float)
    # Include entries ON or BEFORE the target date
    relevant_entries = [
        entry for entry in ledger
        if datetime.strptime(entry['EffectiveDate'], '%Y-%m-%d').date() <= target_date
    ]
    for entry in relevant_entries:
        ownership[entry['OwnerName']] += entry['ChangeInSharePercentage']
    return {name: round(share, 4) for name, share in ownership.items() if abs(share) > 1e-9} # Use fractions here

def visualize_sankey_flow(ledger, filename="sankey_flow.html"):
    """Generates an interactive Plotly Sankey diagram showing ownership flow."""

    if not ledger:
        print("Ledger data is empty.")
        return

    # --- 1. Prepare Data ---
    # Get unique sorted dates
    dates = sorted(list(set(datetime.strptime(e['EffectiveDate'], '%Y-%m-%d').date() for e in ledger)))
    # Add a "start" point before the first real date
    start_date = dates[0] - timedelta(days=1) # Pseudo-date before first transaction
    all_dates = [start_date] + dates

    # Calculate ownership state after each date
    states = {d: get_ownership_at_date(ledger, d) for d in all_dates}

    # Create labels for nodes (Owner@Date) and map them to indices
    node_labels = []
    node_map = {}
    node_colors = [] # Optional: color nodes by owner
    all_owners = sorted(list(set(e['OwnerName'] for e in ledger)))
    owner_color_map = {name: f'hsl({i * 360 / len(all_owners)}, 70%, 70%)' for i, name in enumerate(all_owners)}


    for i, date in enumerate(all_dates):
        date_str = date.strftime('%Y-%m-%d')
        state = states[date]
        for owner, share in state.items():
            if share > 1e-9:
                label = f"{owner}@{date_str}"
                if label not in node_map:
                    node_map[label] = len(node_labels)
                    node_labels.append(label)
                    node_colors.append(owner_color_map.get(owner, 'grey')) # Assign color


    # Create links (source, target, value)
    link_source = []
    link_target = []
    link_value = []
    link_label = [] # Hover text

    # Iterate through time periods (between consecutive dates)
    for i in range(len(all_dates) - 1):
        date_prev = all_dates[i]
        date_curr = all_dates[i+1]
        date_prev_str = date_prev.strftime('%Y-%m-%d')
        date_curr_str = date_curr.strftime('%Y-%m-%d')

        state_prev = states[date_prev]
        state_curr = states[date_curr]

        # Deeds effective on date_curr
        deeds_at_curr = [e for e in ledger if datetime.strptime(e['EffectiveDate'], '%Y-%m-%d').date() == date_curr]

        for owner, share_prev in state_prev.items():
            if share_prev < 1e-9: continue # Skip if owner had no share previously

            owner_label_prev = f"{owner}@{date_prev_str}"
            if owner_label_prev not in node_map: continue # Should exist if share_prev > 0
            source_idx = node_map[owner_label_prev]

            share_curr = state_curr.get(owner, 0.0)

            # A: Persistence Flow (Share kept by the owner)
            persisting_share = min(share_prev, share_curr)
            if persisting_share > 1e-9:
                 owner_label_curr = f"{owner}@{date_curr_str}"
                 if owner_label_curr in node_map: # Target node must exist
                    target_idx = node_map[owner_label_curr]
                    link_source.append(source_idx)
                    link_target.append(target_idx)
                    link_value.append(persisting_share * 100)
                    link_label.append(f"{owner} keeps {persisting_share*100:.2f}%")

            # B: Outgoing Transfer Flow (Share transferred *out* by this owner)
            transferred_share = share_prev - persisting_share # Amount owner gave away
            if transferred_share > 1e-9:
                # Find deeds where this owner was a grantor on date_curr
                grantor_deeds = [d for d in deeds_at_curr if d['OwnerName'] == owner and d['ChangeInSharePercentage'] < -1e-9]
                for deed in grantor_deeds:
                    deed_id = deed['DeedID']
                    # Find grantees in the SAME deed
                    grantees = [g for g in deeds_at_curr if g['DeedID'] == deed_id and g['ChangeInSharePercentage'] > 1e-9]
                    for grantee_entry in grantees:
                        grantee_name = grantee_entry['OwnerName']
                        share_gained_by_grantee = grantee_entry['ChangeInSharePercentage']

                        grantee_label_curr = f"{grantee_name}@{date_curr_str}"
                        if grantee_label_curr in node_map: # Target node must exist
                            target_idx = node_map[grantee_label_curr]
                            link_source.append(source_idx)
                            link_target.append(target_idx)
                            link_value.append(share_gained_by_grantee * 100) # Value is what grantee received
                            link_label.append(f"{owner} -> {grantee_name} ({deed.get('DocRef', deed_id)}): {share_gained_by_grantee*100:.2f}%")


    # --- 2. Create Plotly Figure ---
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = node_labels,
          color = node_colors # Assign colors to nodes
        ),
        link = dict(
          source = link_source,
          target = link_target,
          value = link_value,
          label = link_label, # Add hover labels to links
          # color = link_colors # Optional: color links
      ))])

    fig.update_layout(title_text="Property Ownership Flow", font_size=10)

    try:
        fig.write_html(filename)
        print(f"Sankey diagram saved as {filename}")
        # fig.show() # Uncomment to automatically open in browser
    except Exception as e:
        print(f"Could not save Sankey diagram. Error: {e}")


# --- Example Usage ---
# Assume ledger_data is loaded

# 1. Calculate final status (optional, for context)
final_ownership = get_final_ownership_status(ledger_data)
print(f"\nFinal Ownership Status (as of {datetime.now().date()}):")
for owner, share in final_ownership.items():
    print(f"- {owner}: {share}%")
print("-" * 20)

# 2. Generate Vertical Graphviz
# visualize_full_flow_vertical(ledger_data, final_ownership.keys(), filename="flow_vertical")

# 3. Generate Sankey Diagram
visualize_sankey_flow(ledger_data, filename="ownership_sankey.html")


Final Ownership Status (as of 2025-04-02):
- Origin: -100.0%
- Marco: 50.0%
- Carlos: 20.0%
- Laura: 20.0%
- Maria: 10.0%
--------------------
Sankey diagram saved as ownership_sankey.html
